In [1]:
from custom_matching_functions import *

# Context
In 2012, Alvin Roth and Lloyd Shapley [won](https://www.nytimes.com/2012/10/16/business/economy/alvin-roth-and-lloyd-shapley-win-nobel-in-economic-science.html) the Nobel Prize in Economics for their work in developing seminal theory related to matching markets, and for applying matching markets to address key challenges related to [organ transplants](XX), [medical residency](https://web.stanford.edu/~alroth/papers/rothperansonaer.PDF), and [public school assigments](XX). In this problem, we'll explore how to use matching markets to allocate these resources (which is often the approach actually used in the real world!). In the process, we hope you'll get a sense for some of the complexity involved in applying matching markets to solve real problems. We also hope you'll use this exercise to critically reflect on the dis/advantages of using matching markets to design systems that allocate important resources like education, healthcare, and professional opportunity.

# Instructions

This problem has 4 parts. To get full credit in each part, you'll need to:

1. Run some code to find a matching that satisfies the desired distributive properties.
2. Write **1-3 sentences** per question, analyzing outputs or plots from the matching.
3. Instead of submitting the notebook, please submit your answers on a separate document such as a Word document. Please be sure to label your answers (e.g. Part 1. (a), Part 1. (b), etc.)

As a quick check, make sure you've filled in every `??`in this notebook! While you do not need to submit the notebook itself, you will need to fill in the missing code to run the analysis. As always, if you have questions, feel free to ask on Slack or in office hours. 

# Part 1: Public School Assignments

In some major US cities -- including New York City and Boston -- school districts use matching markets in order to assign students to schools. These cities avoid assignments based on location, in order to try to eliminate racial and socioeconomic [disparities](https://www.gse.harvard.edu/news/ed/19/01/good-schools-close-home) in access to quality schools. 

We'll look at the example of [Boston Public Schools](https://mgetit.lib.umich.edu/go/8697971). The school district's goal is to assign each of the 100* incoming middle schoolers (current 5th grade students) to one of 5 middle schools in the Boston Public School system. Each student ranks each of the five schools, and each school has a certain number of spots open for incoming students. The number of open spots is equal to the number of students. The district wants to assign students to schools in a way that, on the whole, maximizes how happy students are with their assignment. Let's see how we can use matching markets* to do this!

_*Note: Of course, Boston Public Schools doesn't do exactly what we're doing here! This is just an example! Also, Boston Public Schools conducts this process for more like 4,300 middle schoolers per year -- we are just looking at 100 students so things don't take as long to run!_

Note: You may be tempted to think about schools as sellers, students as buyers, and rankings as valuations in this public school matching market. But there are a few differences between the matching markets we studied in class and this example that we'll need to account for:

- **Difference 1:** In class we said that sellers are only selling one item each, but here schools have multiple spots open. Let's say, among the five schools, sch1 has 20 open positions, sch2 has 15, sch3 has 15, sch4 has 20, and sch5 has 30 -- for a total of 100 open positions. In this matching market, we'll have 100 sellers: one corresponding to each position open in each school, so for instance 'sch1_001' (position 1 in school 1) and 'sch_002' (position 2 in school 1) are each considered different sellers. This makes it so each seller is only selling one item, and the number of buyers (students) is equal to the number of sellers (positions in schools).
- **Difference 2:** Notice that positions in each school are identical (i.e., there is no difference to the student if they are matching to sch1_pos001 vs. sch1_pos002). Therefore, for instance, if stu1 ranks sch1 highest, then we assume they're giving a rank of 1 (and the same valuation) to all 20 positions in sch1.
- **Difference 3:** In class, buyers gave a higher valuation to items they liked better, but here a ranking of 1 indicates a higher preference than a ranking of 5 even though 1 < 5. Let's say student 1 ranks the five schools in the following order: sch1 (highest preference), sch4, sch2, sch5, sch3 (lowest preference). In our data, we'll assign a valuation of 5 to all 20 positions in sch1, a valuation of 4 to all positions in sch4, and so on (i.e., the valuation is the inverse of the student's ranking of the school, and the valuations are the same in all positions within the school).


To see how this works, let's read in a dataframe containing a list of potential (school position, student) pairs.

_Note: This data is synthetic since Boston School District doesn't release the actual rankings._

In [2]:
school_ranks = pd.read_csv('Data/school_rankings_synthetic_data.csv')

Let's interpret the very first row of this dataframe.

- **Student**: stu1 is one of the 5th graders looking for a spot in a middle school.
- **SchoolPosition**: sch1_pos001 is one of the positions available in sch1 (one of the five schools)
- **StudentsRanking**: sch1 is stu1's first (top) choice.
- **MatchQuality**: This is a number corresponding to how well sch1_pos001 works for stu1; remember, this number is highest when the ranking is 1 and lowest when the ranking is 5.
- **ClosestSchool**: stu1 lives closest to sch1
- **Race:** stu1 is white (options are Black, Latinx, white -- unfortunately other racial/ethnic groups had too few numbers for the purposes of our analysis)

In [3]:
school_ranks.head(1)

,Student,SchoolPosition,StudentsRanking,MatchQuality,ClosestSchool,Race
0,stu1,sch1_pos001,1,5,sch1,White


Here are `stu1`'s rankings and match quality for positions in all five schools.

In [4]:
school_ranks[(school_ranks.Student=='stu1') & 
             (school_ranks.SchoolPosition.str.contains('pos001'))]

,Student,SchoolPosition,StudentsRanking,MatchQuality,ClosestSchool,Race
0,stu1,sch1_pos001,1,5,sch1,White
20,stu1,sch2_pos001,3,3,sch1,White
35,stu1,sch3_pos001,2,4,sch1,White
50,stu1,sch4_pos001,4,2,sch1,White
70,stu1,sch5_pos001,5,1,sch1,White


As explained before, you can see that `stu1` has the same ranking (and, therefore, match quality) for positions 1 and 2 (and all positions) within the same school.

In [5]:
school_ranks[(school_ranks.Student=='stu1') & 
             ((school_ranks.SchoolPosition.str.contains('pos001') |
               school_ranks.SchoolPosition.str.contains('pos002')))]

,Student,SchoolPosition,StudentsRanking,MatchQuality,ClosestSchool,Race
0,stu1,sch1_pos001,1,5,sch1,White
1,stu1,sch1_pos002,1,5,sch1,White
20,stu1,sch2_pos001,3,3,sch1,White
21,stu1,sch2_pos002,3,3,sch1,White
35,stu1,sch3_pos001,2,4,sch1,White
36,stu1,sch3_pos002,2,4,sch1,White
50,stu1,sch4_pos001,4,2,sch1,White
51,stu1,sch4_pos002,4,2,sch1,White
70,stu1,sch5_pos001,5,1,sch1,White
71,stu1,sch5_pos002,5,1,sch1,White


**a) In 1-2 sentences, describe what goal the Boston Public School district is trying to accomplish.** _There are many right answers._

(write your answer on a separate document)

**b) How would you describe Boston's public school assignments as a matching market?** _Hint: When we talked about matching markets we always have buyers, sellers, and buyer valuations. Which columns in the dataframe correspond to the buyers, sellers and valuations?_

In [6]:
seller_column_name_1 = ?? #fill this in: column name for the seller in school_ranks
buyer_column_name_1 = ?? #fill this in: column name for the buyer in school_ranks
valuation_column_name_1 = ?? #fill this in: column name for the valuation in school_ranks


SyntaxError: invalid syntax (Temp/ipykernel_16300/2542926052.py, line 1)

Now run this code to find the optimal matching of students to schools.

In [7]:
school_match, optimal_school_matching, school_market_clearing_prices =\
find_optimal_matching(school_ranks, seller_column_name_1, buyer_column_name_1, valuation_column_name_1)

NameError: name 'seller_column_name_1' is not defined

And verify that all students were assigned to a school, and all spots in all schools are filled.

In [ ]:
count_matched_and_unmatched(graph=school_match, matching=optimal_school_matching)

**c) What is the social welfare of the optimal matching you obtained in subpart 1.b? At a conceptual level, what does the social welfare of this matching represent?** _There are many right answers._

(write your answer on a separate document)

In [ ]:
social_welfare(graph=school_match, matching=optimal_school_matching)

**d) Run the code below to examine the relationship between how highly students tend to rank each school (x-axis; 1 is highest and 5 is lowest) and the school's market-clearing price (y-axis); the table and scatterplot show the same information. How would you describe the relationship, based on this chart? Why do you think the market-clearing prices exhibit these trends?** _There are many right answers._

(write your answer on a separate document)

In [ ]:
plot_school_prices(school_ranks, school_market_clearing_prices)

**e) Run the code below to compare the matching market you created in subpart (b) to see what happens when you assign students to school by highest ranking (described below*). These plots are comparing how highly students ranked the school that they were ultimately matched to.** 

**How does students' rankings of their assignments change when you assign by matching market rather than by highest rank? Based on these graphs, which approach (matching market or assigning by highest rank) do you think is better? Briefly explain.** _There's no right or wrong answer; just share your opinions. Moreover, it's okay to not know what your opinion is; in this case you can share what makes it hard to answer this question._

_*How Assignment by Highest Rank Works: The school district first tries to give all students their top-ranked choice, breaking ties randomly (e.g., if 60 students ranked sch1 as their top choice but there are only 20 spots open, then they'd randomly choose 20 of those 60 students to be in sch1). Then they give any unassigned students their second choice if that school still had spots open, breaking ties randomly. And so on until, all students are assigned to a school. Before Boston started using a matching market, this is how they used to assign students to schools._ 

(write your answer on a separate document)

In [ ]:
plot_public_school_matching(school_ranks, matching = optimal_school_matching)

**f) Often, school districts will send students to the school that's closest to them. As described previously, Boston Public Schools instead uses matching markets in an attempt to eliminate racial [disparities](https://www.gse.harvard.edu/news/ed/19/01/good-schools-close-home) in access to quality schools (i.e., schools with high test scores, low student:teacher ratios, etc.). If you run the code below, you can see* that there are large racial disparities when we assign students to the closest school; there are still disparities when students are assigned by the matching market from subpart (b).** 

**Why do you think large racial disparities may exist when we assign students to the closest school? Have the racial disparities gotten bigger, smaller or remained the same when we assign students using our matching market?Why might the matching market have this effect?** _There are many right answers here!_

_*How to interpret this plot: The blue bars show the average quality of schools that Black students are assigned to; orange bars for Latinx students; green bars for white students. In the bars on the right (assigned to closest school), Black and Latinx students are assigned to much lower quality schools than white students. In the bars on the left (assigned via matching market), Black and Latinx students still haven't caught up to White students (i.e., disparity is not eliminated)._

(write your answer on a separate document)

In [ ]:
plot_school_by_race(school_ranks, matching = optimal_school_matching)

**g) Looking at the plot from subpart (f), we can see that Black and Latinx students are assigned to lower quality schools than white students by the matching market from (b). One reason for these racial disparities could be Boston's policy to minimize transportation costs: They do not offer transportation for students traveling more than 25 miles from home to school.\*** 

**How might this policy contribute to creating these racial disparities in the assignments by the matching market? You should explain what direct effects the policy has on people, and how that would cause the matching market to have disparate outcomes. Boston cannot afford to eliminate this policy, but what other actions by the city may help further reduce racial disparities in school assignments? Explain.** _There's no right or wrong answer; just share your opinions. Moreover, it's okay to not know what your opinion is; in this case you can share what makes it hard to answer this question._

_\*This is not Boston's actual policy, but it's pretty close to [what](https://www.gse.harvard.edu/news/ed/19/01/good-schools-close-home) they do and has a similar impact!_

(write your answer on a separate document)

**h) In last week's problem set, we learned about baseline-maximizing matchings. That is, a matching that would ensure that no student gets assigned to a school they're too unhappy with. Run the code below\*. Based on the code's output\* and the plot from subpart (e): What is the highest baseline a matching can have (assuming that all students have to go to some school)? Is the optimal matching we found in subpart (c) also baseline-maximizing? Explain your answers to both questions.**

_\*This code prints out whether there is a perfect matching of students to schools if we only match students to schools they ranked as their top x choices. For instance, the first line tells us that it is not possible to match all students to only their top-ranked school. The second line tells us that it is not possible to match all students to only their top two schools; e.g., more students had sch1 and sch2 in their top two than these two schools have capacity to enroll. However, the third line tells us that **is** possible to match all students to only their top three schools; and so on. In this problem, you have to think about: What does all of this information tell you about what is the highest possible baseline of this matching? And has the optimal matching in (c) achieved that highest possible baseline?_

(write your answer on a separate document)

In [ ]:
print("Is there a perfect matching of students to schools if we only match students to schools they ranked:")
for i in range(1,6):
    print("  -",str(i),"and Higher?",
          find_perfect_matching(school_ranks[school_ranks.StudentsRanking<=i], 
                                seller_column_name_1, buyer_column_name_1)[1])

_Note: The actual process Boston uses to get rankings from students is a little more complex! If you're interested, you can read about it, and some of its critiques, [here](https://www.gse.harvard.edu/news/ed/19/01/good-schools-close-home) -- but reading this article is **not** required for the assignment._

# Part 2: Medical Residency 

Another famous example of matching markets in action is medical residency. When they finish med school, students pick a specialty (e.g., anesthesiology, pediatrics) and look to become a resident physician in that area. Each year, hospitals decide which departments they need resident physicians in and interview senior med students for these spots. To help coordinate these placements, [The National Residency Matching Program](https://www.nrmp.org/) (NRMP) has residents rank hospitals they'd be willing to work in, and hospitals rank residents they'd be willing to work with. 

In this problem, we'll look at NRMP matching 189 anesthesiology residents to 189 open positions across 55 hospitals in 2022. If a resident does not want to be matched to a hospital, they won't include them in their ranking and vice versa. On average, residents usually rank ~10-15 programs, signaling that they would only go to these 10-15 places. Once NRMP has these rankings, they match residents to open spots in a way that best respects everyone's preferences. Let's take a look at how this process could* work.

_*Note: Of course, NRMP doesn't do exactly what we're doing here! This is just an example!_

Again, there are several differences between the matching market in this example and the ones in the textbook:

- **Differences 1-3:** Same as Part 1. Each opening (not each hospital) is a seller, rankings are the same for each open position in the same hospital, and valuations are the inverse of rankings.
- **Difference 4:** In class, buyers had valuations for each seller's items, but here both buyers (residents) _and_ sellers (open positions) are providing rankings. We account for this by creating a combined _match quality_ score, which is higher when both parties rank each other highly and lower when both give each other low rankings. This _match quality_ score becomes the valuation that we are trying to maximize in the matching.
- **Difference 5:** As discussed in class, there are instances where buyers may not want every seller's item. If the resident did not rank the hospital or the hospital did not rank the resident, we assume that the valuation from that matching would be _negative_ so it's better to leave them unmatched than to match them. Note that the `networkx` matching function knows how to handle missing valuations, so we do not need to add these pairs to the graph or put in a negative number.

To see what this looks like in practice, let's read in a dataframe containing a list of potential (hospital, resident) pairs -- meaning the hospital ranked the resident and the resident ranked the hospital.

_Note: This data is synthetic since NRMP doesn't release the actual rankings -- but we tried to create the data in a way that matches some characteristics of the rankings which NRMP published [here](https://www.nrmp.org/wp-content/uploads/2022/03/Advance-Data-Tables-2022-FINAL.pdf)._

In [ ]:
residency_ranks = pd.read_csv('Data/resident_matches_synthetic_data.csv')

Let's interpret the very first row of this dataframe.

- **HospitalPosition**: hosp001_pos1 is one of the anesthesiology positions open at hospital #1.
- **Resident**: res081 is one the residents looking for an anesthesiology residency.
- **HospitalsRank**: res081 is hosp_001's 21st choice (1 is highest). 21st may seem low but remember the hospital is signaling their willingness to work with the resident by including them in the ranking at all! They could easily have left this resident off their ranking if they didn't want to work with them.
- **ResidentsRank**: hosp001 is res081's 13th choice.
- **MatchQuality**: This is a number corresponding to how well the match works for both sides; if both ranked each other highly the number is high, if both ranked each other low the number is low. We made this number for you -- you don't have to worry about the formula used beyond what's been described in this document.

In [ ]:
residency_ranks.head(1)

The goal is to match residents to open _positions_, and a resident can get any of the open positions at a hospital where there is mutual interest. For instance, resident _res001_ ranked 10 hospitals. In the data below, we see that just one of these hospitals (_hosp006_) also ranked _res001_. That means _res006_ has a shot at getting any of the 3 positions open at _hosp006_.

In [ ]:
residency_ranks[(residency_ranks.Resident=="res001")]

**a) In 1-2 sentences, describe what goal NMRP is trying to accomplish.** _There are many right answers._

(write your answer on a separate document)

**b) How would you describe NRMP's residency placements as a matching market?** _Hint: When we talked about matching markets we always have buyers, sellers, and buyer valuations. Which columns in the dataframe correspond to the buyers, sellers and valuations?_

In [ ]:
seller_column_name_2 = ?? #fill this in: column name for the seller in school_ranks
buyer_column_name_2 = ?? #fill this in: column name for the buyer in school_ranks
valuation_column_name_2 = ?? #fill this in: column name for the valuation in school_ranks


**c) The following code prints `True` if there is a perfect matching of residents to open positions, and `False` otherwise. Based on these results, do you think that all residents will get a position in an optimal matching? Explain.** 

(write your answer on a separate document)

In [ ]:
_, has_perfect_match_residency, perfect_matching_residency, constricted_set_residency =\
    find_perfect_matching(residency_ranks, seller_column_name_2, buyer_column_name_2)
print(has_perfect_match_residency)

**d) 118 residents are matched to open positions in hospitals, while 71 residents and openings remain unfilled. We started with the same number of incoming residents and open positions. Based on the outputs below, give two reasons why some remained unmatched.**

(write your answer on a separate document)

In [ ]:
# Run this code to find the optimal matching of residents to hospitals.
residency_match, optimal_residency_matching, hospital_market_clearing_prices =\
find_optimal_matching(residency_ranks, seller_column_name_2, buyer_column_name_2, valuation_column_name_2)

In [ ]:
# count matched and unmatched in optimal matching
count_matched_and_unmatched(graph=residency_match, matching=optimal_residency_matching)

In [ ]:
# Note: the number of unmatched residents in the optimal matching is just slightly more than 
# the number of neighbors of the constricted set in the perfect matching 
len(constricted_set_residency[1])

**e) Run the following code to analyze how many residents and hospital jobs are matched. You can see that stronger applicants (first plot) and more reputable hospitals (second plot) are more likely to get matched. What effect do you think this would have on inequities in access to quality healthcare? Briefly explain.** _There's no right or wrong answer; just share your opinions. Moreover, it's okay to not know what your opinion is; in this case you can share what makes it hard to answer this question. Remember, we don't expect you to write more than 1-3 sentences!_

(write your answer on a separate document)

In [ ]:
plot_residency_match_rates(optimal_residency_matching, outcome='quality')

**f) Run the following code to analyze how many residents and hospital jobs are matched. You can see that residents who rank more programs are more likely to get matched (first plot); also, hospitals who ranked more applicants per spot tend to have more positions filled (second plot). Why do you think this is the case? As an applicant, could there ever be a situation where ranking more schools could get you a worse outcome than ranking fewer schools? Provide a high-level, intuitive explanation for your answers.** _There are many right answers! And remember, we don't expect you to write more than 1-3 sentences!_

(write your answer on a separate document)

In [ ]:
plot_residency_match_rates(optimal_residency_matching, outcome='num_ranked')

**g) Run the code below to examine the relationship between a hospital's reputation (x axis; higher is better) and its market-clearing price (y axis). How would you describe the relationship, based on this plot? At a conceptual level, why do you think the market-clearing prices exhibit this trend?** _There are many right answers._

(write your answer on a separate document)

In [ ]:
plot_hospital_prices(market_clearing_prices = hospital_market_clearing_prices)

**h) If NRMP didn't exist, then hospitals would directly make offers to the top residents they interviewed.* Run the following code, which prints, for both matchings, the social welfare and the rankings of the matched pairs. At a conceptual level, what does the social welfare of these matchings represent? Based on the graphs, why is the social welfare higher with the optimal matching than when hospitals make offers on their own?** _There are many right answers._

_*How it would work if hospitals directly made offers to residents: Hospitals would give a job offer to their top-ranked resident for each spot. Then residents who received an offer would pick their top-ranked offer. Then, for any remaining unmatched spots, hospitals would make offers to their top-ranked remaining residents. And so on._

(write your answer on a separate document)

In [ ]:
plot_residency_rankings(residency_ranks, graph=residency_match, matching=optimal_residency_matching)

_Note: The actual process NRMP uses to match residents to hospitals is a little more complicated! In particular, they look for a stable matching rather than an optimal matching. If you're interested, you can watch a video about it [here](https://www.nrmp.org/intro-to-the-match/how-matching-algorithm-works/) -- but this piece is **not** required for the assignment and you do not need to know about stable matching on the exams (only perfect and optimal matching)._

# Part 3: Kidney Exchange

Each year, hundreds of thousands of Americans are on the waiting list to receive an organ transplant. This waiting list is managed by the [United Network for Organ Sharing](https://unos.org/transplant/how-we-match-organs/) (UNOS). When organ donations come in (from deceased or living donors), UNOS first identifies which members of the waiting are candidates to receive those organs (e.g., same blood type, high enough histocompatibility). Then they give the organs to the candidate whose life will most be extended by the transplant -- e.g., if there are two candidates for a kidney, one will live 10 more years with a transplant and one will live 30 more years, then the second person gets the kidney. Of course, there are multiple kidneys coming in at a time and some people may be a candidate to receive more than one of them, so the assignment process can get a little complicated. 

In this problem, we'll simulate the type of decision that UNOS has to make about kidney matching. Let's say they have received 100 kidneys in the last 36h and have to decide how to assign them to the 499 people on the waiting list whose blood/tissue types make them a candidate to receive one. Let's explore how matching markets play a role here!

Once again, there are a few differences between this example and the matching markets we saw in the textbook:

- **Difference 1:** In this example, there are more buyers (people on the waiting list) than sellers (kidneys). In order to handle this scenario, the book says to add in several 'placeholder' sellers for whom all buyers have negative valuation (this last part ensures that no buyer actually gets matched to one of these dummy sellers). Here, the `networkx` matching function knows how to handle uneven numbers of sellers and buyers, so we do not need to add these extra sellers to the graph or put in the negative valuations.
- **Difference 2:** In this example, a buyer may not want every seller's item. If someone on the waiting list is not a candidate to receive a kidney, we assume that the valuation from that matching would be negative so it's better to leave them unmatched than to match them. Again, the `networkx` matching function knows how to handle missing valuations, so we do not need to add these pairs to the graph or put in a negative valuations.

To see how this works, let's read in a dataframe containing a list of potential (kidney, candidate) pairs -- meaning a donated kidney and a member of the waiting list who's a candidate to receive it.

_Note: This data is synthetic since UNOS doesn't release the actual donor/candidate data -- but we tried to create the data in a way that matches some characteristics of the rankings which UNOS published [here](https://unos.org/solutions/research-data-analytics-transplant/) and [here](https://www.kidney.org/atoz/content/BloodTests-for-Transplant)._

In [ ]:
kidney_exchange = pd.read_csv('Data/kidney_matches_synthetic_data.csv')

Let's interpret the very first row of this dataframe.

- **Donor**: don001 has donated a kidney.
- **Candidate**: cand002 is on the waiting list to receive a kidney and is a candidate to receive don001's kidney (i.e., they have the same blood type, tissues are compatible).
- **Age**: cand002 is 35 years old
- **HasMedicalConditions**: cand002 does not have other medical conditions
- **SurvivalWithoutTransplant**: cand002 is expected to live 21 months without a transplant
- **LifeExtendedWithTransplant**: cand002 is expected to live 41 more years if they receive don001's kidney. Note that this may be different from kidney to kidney (e.g., if the kidney is more compatible the ) and, of course, also different between people on the waiting list (e.g., an older person or someone with another medical condition is expected to have less life ahead of them).
- **WaitingListOrder**: cand002 was the 116th person on the waiting list to join (i.e., 115 other people have been waiting longer than cand002 to receive a kidney).


In [ ]:
kidney_exchange.head(1)

cand002 is a candidate to receive three potential kidneys

In [ ]:
kidney_exchange[kidney_exchange.Candidate=='cand002']

don001's kidney is a match for 53 different candidates

In [ ]:
print(kidney_exchange[kidney_exchange.Donor=='don001'].shape)
kidney_exchange[kidney_exchange.Donor=='don001'].head()

**a) In 1-2 sentences, describe what goal UNOS is trying to accomplish.** _There are many right answers._

(write your answer on a separate document)

**b) How would you describe the kidney exchange as a matching market?** _Hint: When we talked about matching markets we always have buyers, sellers, and buyer valuations. Which columns in the dataframe correspond to the buyers, sellers and valuations?_

In [ ]:
seller_column_name_3 = ?? #fill this in: column name for the seller in school_ranks
buyer_column_name_3 = ?? #fill this in: column name for the buyer in school_ranks
valuation_column_name_3 = ?? #fill this in: column name for the valuation in school_ranks


Now run this code to find the optimal matching of students to schools.

In [ ]:
kidney_match, optimal_kidney_matching, donor_market_clearing_prices =\
find_optimal_matching(kidney_exchange, seller_column_name_3, buyer_column_name_3, valuation_column_name_3)

**c) Run the code below and verify that 1 kidney was not given to anyone on the waiting list, and 400 candidates were not given a kidney. Why do you think some kidneys were not matched?**

(write your answer on a separate document)

In [ ]:
count_matched_and_unmatched(graph=kidney_match, matching=optimal_kidney_matching)

**d) Run the code below to plot the market-clearing price of each kidney by the donor's blood type. Which blood types tend to have higher market-clearing prices? Lower market-clearing prices? Why do you think this is the case?** _There are many right answers!_

_Hint: [recall](https://www.kidney.org/atoz/content/BloodTests-for-Transplant) that:_
- _Donors with blood type O can donate to candidates with blood types O, A, B, and AB_
- _Donors with blood type A can donate to candidates with blood types A and AB_
- _Donors with blood type B can donate to candidates with blood types B and AB_
- _Donors with blood type AB can donate to candidates with blood type AB only_

_How could this be affecting the market-clearing prices?_

(write your answer on a separate document)

In [ ]:
plot_donor_prices(market_clearing_prices = donor_market_clearing_prices)

**e) How many years of life have been extended as a result of these kidney transplants? We have created a function that calculates this for you, but this function calculates the equivalent of one of the matching properties that we learned about in class. Which property is this and why is it equivalent?**

(write your answer on a separate document)

In [ ]:
life_extended(kidney_match, optimal_kidney_matching)

**f) Run the code below to analyze who receives a match in the matching market. As you can see, people who are older (first plot), with medical problems (second plot), and with lower survival (third plot) are less likely to get a kidney. Why do you think the matching market you set up in part (b) would produce this outcome? Do you think this is fair? Explain.** _For the question on fairness, there's no right or wrong answer; just share your opinions. Moreover, for the question on fairness, it's okay to not know what your opinion is; in this case you can share what makes it hard to answer this question._

(write your answer on a separate document)

In [ ]:
plot_kidney_exchange(kidney_exchange, matching = optimal_kidney_matching)

**g) Of course, there are ways other than matching markets to assign kidneys to people! For instance, UNOS could give priority to people who i) are higher on the waiting list or ii) who would have lower survival without the transplant. Let's see how these two options compare against our matching market. For each of these three kidney assignment strategies, the code below prints out (1) how many years of life would be extended by the respective transplants (similar to e), and (2) graphs showing whether there are disparities in who gets assigned a kidney (similar to f).** 

**Compare and contrast the outputs for the different assignments of kidneys to donors (matching market, waiting list order, lowest survival). Why do you think these differences may occur, based on your understanding of how the three procedures work?** _There are many right answers!_

(write your answer on a separate document)

In [ ]:
plot_kidney_markets(kidney_exchange, kidney_match, optimal_kidney_matching)

**h) Based on the outputs from subpart (g), which of the three kidney assignment strategies (matching market, waiting list order, lowest survival) do you think is most fair? How do you think UNOS should decide which of these three procedures to use -- and which stakeholders should get a voice in the decision-making process?** _There's no right or wrong answer; just share your opinions. Moreover, it's okay to not know what your opinion is; in this case you can share what makes it hard to answer this question._

(write your answer on a separate document)

# Part 4: Reflection

To conclude the assignment, let's reflect on what we just did. To get full credit just write 1-3 sentences answering the following questions. There's no right or wrong answer to any of these questions!

**a) What are one or two things that make it difficult to apply matching markets to allocate scarce societal resources like public schooling, medical residencies, and kidneys? Write 1-3 sentences sharing your general impressions accompanied by at least one specific example of what you mean.**

(write your answer on a separate document)

**b) What do you think are some potential advantages of using a matching market to allocate important resources like public schooling, medical residencies, and kidneys?**

(write your answer on a separate document)

**c) What do you think are some potential disadvantages of using a matching market to allocate important resources? Can the issue(s) you're raising be addressed by changing the valuations? Or are they problem(s) inherent to using matching markets?**

_If you're interested, you can check out some critiques of the [public school assignments](https://www.chicagobooth.edu/review/when-students-are-matched-schools-who-wins), [residency matching](https://www.forbes.com/sites/davidwhelan/2012/10/15/alvin-roth-receives-economics-nobel-for-flawed-residency-match-system/?sh=701bec40376c), and [kidney exchange](https://www.gsb.stanford.edu/insights/beautiful-application-using-economics-make-kidney-exchanges-more-efficient-fair) work (though you may want to ignore the weird marriage metaphors)! But you **don't** have to read these articles to answer this question. Just share your own thoughts._

(write your answer on a separate document)

**d) In many of these examples, a goal of using the matching market was to eliminate several inequities in the distribution of important resources (e.g., removing location-related racial disparities in school assignments, preventing only top med school graduates from getting residency offers). To what extent do you think a matching market is necessary to achieve this goal? To what extent is it sufficient (on its own) to achieve this goal? If it's not sufficient, what are some other things that are required to achieve this goal? Explain your thought process.**

(write your answer on a separate document)

You're done! Congrats! You'll need to submit this document as a PDF. To do that go to File > Print Preview > Save as PDF. Then use a PDF merger tool (there are many available for free online) to merge the PDF from this problem into your PDF for the rest of the problem set.